# テーマ2

# NG_特定のカラムだけにする

In [6]:
import joblib
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# 例としてデータロードとモデル訓練を行います
uploaded = files.upload()

# CSVファイルを読み込む
df = pd.read_csv('train.csv')

# ヘッダー行から列名を動的に取得する
column_names_to_extract = ['1', '2', '3', '4', '5', '6', '15', '16', '17', '18', '23', '24']  # 必要な列名をリストで指定

# 動的に取得した列名を使ってデータを抽出する
extracted_columns = df[column_names_to_extract]

# 抽出したデータを新しいCSVファイルとして保存する
extracted_columns.to_csv('extracted_data.csv', index=False)

# ファイルが保存されたことを確認するためにメッセージを表示
print("Extracted data has been saved to extracted_data.csv")

Saving train.csv to train (5).csv


KeyError: "None of [Index(['1', '2', '3', '4', '5', '6', '15', '16', '17', '18', '23', '24'], dtype='object')] are in the [columns]"

# データ補完(湿度の補完) 降水量*50

In [14]:
import joblib
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# 例としてデータロードとモデル訓練を行います
uploaded = files.upload()

# Read the CSV file
df = pd.read_csv('test_extracted_train.csv')

# Handle rows where the 5th column is empty
df['Humidity3pm'] = df.apply(
    lambda row: (int(row.iloc[3]) * 50) % 101 if pd.isnull(row.iloc[4]) else row.iloc[4],
    axis=1
)
# Filter rows where the 5th column was filled (to avoid NaN values)
df = df[~pd.isnull(df['Humidity3pm'])]

# Select the columns you want to save
columns_to_save = ['id', 'Date', 'Location', 'Rainfall', 'Humidity3pm', 'RainToday', 'RainTomorrow']  # Replace with actual column names

# Save the processed data to a new CSV file
df[columns_to_save].to_csv('hokann_test_extracted_train.csv', index=False)

Saving test_extracted_train.csv to test_extracted_train (5).csv
